In [1]:
#CLUSTER GORDIAS LIS 
import pyTigerGraph as tg
import pandas as pd
import gradio as gr
hostName = "https://2cc2f8bde8df444bb60c6fb83491bb8c.i.tgcloud.io"
graphName = "VWG"
secret = "m2p8nba0uab7dtthbn4o1b30r8tqgg9a"
userName = "user_1"
password = "A1z2e3r4*"
conn = tg.TigerGraphConnection(host=hostName, graphname=graphName, username=userName, password=password)
conn.getToken(secret)# tigergraph_insights_map = "https://tools.tgcloud.io/insights/app/qepJkoYLXfcWTB4d5ExgVT/page/4NjQNvfLhhTXHgQZBkyxtM/widgetShare/6wci2PjrXvzciTfmrH9pQY?domain=d6e9eef375704c8893b47bdc6132b082.i&orgName=lis-gordias&clusterid=507badf1-6ed3-4a79-9aaa-82489ed609ef&TigerGraphToken=e4120fbd-1fb5-4383-a727-33fb7032eb53"


c:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\.venv\Lib\site-packages\pyTigerGraph\common\auth.py:130: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  datetime.utcfromtimestamp(


('eh4blq28panjgfqifgk2pqi9mb5bflko', 1733994529, '2024-12-12 09:08:49')

# Tab 0 (cargar datos)

In [2]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
atributes_nodes = {}
atributes_edges = {}

# Helper functions
def path_taken(results):
    # Extrae el conjunto de aristas desde el diccionario de resultados.
    edges = results[2]['@@display_edge_set']
    
    # Inicializa una variable para almacenar el nodo inicial.
    node_initial = None
    
    # Crea un conjunto de identificadores de nodos a los que se dirige cada arista.
    edge_to_ids = {edge['to_id'] for edge in edges}
    
    # Busca el nodo inicial (el nodo que no es el destino de ningún arista).
    for edge in edges:
        if edge['from_id'] not in edge_to_ids:
            node_initial = edge['from_id']
            break
    
    # Si no se encuentra un nodo inicial, lanza una excepción.
    if node_initial is None:
        raise ValueError("initial node cannot be determined.")
    
    # Inicializa el nodo actual como el nodo inicial y crea una lista para el camino.
    current_node = node_initial
    path = [current_node]
    
    # Recorre los bordes para construir el camino desde el nodo inicial.
    while True:
        next_node = None
        
        # Busca el siguiente nodo en el camino basado en el nodo actual.
        for edge in edges:
            if edge['from_id'] == current_node:
                next_node = edge['to_id']
                break
        
        # Si no se encuentra el siguiente nodo, termina el bucle.
        if next_node is None:
            break
        
        # Añade el siguiente nodo al camino y actualiza el nodo actual.
        path.append(next_node)
        current_node = next_node
    
    # Crea un diccionario de nodos mapeando cada nodo a sí mismo.
    nodes = {node_name: node_name for node_name in path}
    
    # Crea una lista ordenada de nodos basada en el camino encontrado.
    ordered_nodes = [nodes[node_name] for node_name in path]
    
    # Devuelve la lista de nodos ordenados que representa el camino tomado.
    return ordered_nodes


def add_or_update_final_nodes_batch(nodes_batch_full, nodes_batch, charge):
    # Crea una lista de ciudades existentes a partir de los nodos en 'nodes_batch_full'.
    existing_cities = [city for city, _ in nodes_batch_full]
    
    # Itera sobre cada nodo en el 'nodes_batch' para actualizar o agregar nodos en 'nodes_batch_full'.
    for node, _ in nodes_batch:
        if node in existing_cities:
            # Si el nodo ya existe en 'nodes_batch_full', busca el índice del nodo para actualizar el stock.
            for i, (city, stock) in enumerate(nodes_batch_full):
                if city == node:
                    # Si es el primer nodo, decrementa el stock por 'charge'.
                    # De lo contrario, incrementa el stock por 'charge'.
                    if i == 0:
                        nodes_batch_full[i] = (city, {"Stock": nodes_batch_full[i][1]["Stock"] - charge})
                    else:
                        nodes_batch_full[i] = (city, {"Stock": nodes_batch_full[i][1]["Stock"] + charge})
            continue
        else:
            # Si el nodo no está en 'nodes_batch_full', agrega nuevas entradas para los nodos de 'nodes_batch'.
            nodes_batch_full.extend([
                (nodes_batch[0][0], {"Stock": nodes_batch[0][1]["Stock"]}),
                (nodes_batch[1][0], {"Stock": nodes_batch[1][1]["Stock"]})
            ])
            # Retorna 'nodes_batch_full' después de agregar las nuevas entradas.
            return nodes_batch_full
    
    # Retorna 'nodes_batch_full' después de actualizar o agregar los nodos.
    return nodes_batch_full

def add_or_update_final_edges_batch(edges_batch_full, edges_batch, charge):
    # Crea una lista de bordes existentes a partir de 'edges_batch_full', considerando ambos sentidos.
    existing_edges = [(edge[0], edge[1]) for edge in edges_batch_full]
    
    # Itera sobre cada borde en 'edges_batch' para actualizar o agregar bordes en 'edges_batch_full'.
    for edge in edges_batch:
        # Verifica si el borde ya existe en 'edges_batch_full' considerando ambos sentidos.
        if (edge[0], edge[1]) in existing_edges or (edge[1], edge[0]) in existing_edges:
            # Si el borde ya existe, busca el índice del borde para actualizar el movimiento diario.
            for i, (origin, destination, movement) in enumerate(edges_batch_full):
                # Compara ambos sentidos del borde para encontrar el índice correcto.
                if (origin, destination) == (edge[0], edge[1]) or (origin, destination) == (edge[1], edge[0]):
                    # Actualiza el movimiento diario del borde sumando 'charge'.
                    edges_batch_full[i] = (origin, destination, {"Daily_movement": edges_batch_full[i][2]["Daily_movement"] + charge})
            continue
        else:
            # Si el borde no existe, agrega una nueva entrada en 'edges_batch_full'.
            edges_batch_full.append((edge[0], edge[1], {"Daily_movement": edge[2]["Daily_movement"]}))
    
    # Retorna 'edges_batch_full' después de actualizar o agregar los bordes.
    return edges_batch_full


def actualize_graph_carga(conn=conn):
    # global aristas_sobrecargadas
    # Obtiene la lista de nodos y aristas desde la conexión proporcionada.
    nodes = conn.getVertices("Nodes")
    aristas = conn.getEdgesByType(edgeType="distribute_to")

    # Inicializa listas para almacenar nodos y aristas con datos escalados.
    nodes_batch_scale = []
    aristas_batch_scale = []
    
    # Inicializa una variable para registrar los estados de nodos y aristas.
    logs_state_nodes_and_arristas = ""

    # Contadores para los diferentes niveles de carga de nodos y aristas.
    node_counts = {"Sobrecargado": 0, "Subcargado": 0, "Optimal": 0}
    arista_counts = {"Sobrecargado": 0, "Subcargado": 0, "Optimal": 0}

    # Itera sobre cada nodo para determinar su nivel de carga.
    for node in nodes:
        node_capacidad = node["attributes"]["Capacity"]
        node_stock = node["attributes"]["Stock"]
        
        # Determina el nivel de carga del nodo basado en su capacidad y stock.
        if node_capacidad == 0:
            nivel_stock = "No definido"
        else:
            if (node_stock / node_capacidad) > 1:
                nivel_stock = "Sobrecargado"
                node_counts["Sobrecargado"] += 1
                logs_state_nodes_and_arristas += f"⚠️ {node['v_id']} es {nivel_stock} \n"
            elif 0.7 < (node_stock / node_capacidad) <= 1:
                nivel_stock = "Optimal"
                node_counts["Optimal"] += 1
            else:
                nivel_stock = "Subcargado"
                logs_state_nodes_and_arristas += f"⚠️ {node['v_id']} es {nivel_stock} \n"
                node_counts["Subcargado"] += 1

        # Añade el nodo con su nivel de carga a la lista de nodos a escalar.
        nodes_batch_scale.append((node["v_id"], {"Carga": nivel_stock}))

    # Itera sobre cada arista para determinar su nivel de carga.
    for arista in aristas:
        capacity_edge = arista["attributes"]["Capacity"]
        daily_movement = arista["attributes"]["Daily_movement"]

        # Determina el nivel de carga de la arista basado en su capacidad y movimiento diario.
        if capacity_edge == 0:
            nivel_arista = "No definido"
        else:
            if (daily_movement / capacity_edge) > 1:
                nivel_arista = "Sobrecargado"
                arista_counts["Sobrecargado"] += 1
                logs_state_nodes_and_arristas += f"⚠️ la arista {arista['from_id']} hasta {arista['to_id']} es {nivel_arista} \n"
                aristas_sobrecargadas.append(arista)
            elif 0.7 < (daily_movement / capacity_edge) <= 1:
                nivel_arista = "Optimal"
                arista_counts["Optimal"] += 1
            else:
                nivel_arista = "Subcargado"
                logs_state_nodes_and_arristas += f"⚠️ la arista {arista['from_id']} hasta {arista['to_id']} es {nivel_arista} \n"
                arista_counts["Subcargado"] += 1

        # Añade la arista con su nivel de carga a la lista de aristas a escalar.
        aristas_batch_scale.append((arista["from_id"], arista["to_id"], {"Carga": nivel_arista}))

    # Actualiza los nodos en la base de datos con los datos escalados.
    conn.upsertVertices("Nodes", nodes_batch_scale)
    
    # Actualiza las aristas en la base de datos con los datos escalados.
    conn.upsertEdges(sourceVertexType="Nodes", targetVertexType="Nodes", edgeType="distribute_to", edges=aristas_batch_scale)

    # Si no se registraron cambios, indica que todos los nodos y aristas están en un estado óptimo.
    if logs_state_nodes_and_arristas == "":
        logs_state_nodes_and_arristas = "Todos los nodos y aristas están en un estado óptimo."

    # Crea un diccionario con los registros de estado y los conteos de nodos y aristas.
    results = {
        "logs": logs_state_nodes_and_arristas,
        "node_counts": node_counts,
        "arista_counts": arista_counts,
        "aristas_sobrecargadas" : aristas_sobrecargadas
    }

    # Retorna el diccionario con los resultados.
    return results


# Main transfer function

def transfert_nodes_and_edges(node_initial, node_final, charge, weight_attribute="Price", nodetype="Nodes", edgetype="distribute_to"):
    global palets_cost, log_list , entregas_and_paths
    # Lanza la función A* para encontrar el camino óptimo entre node_initial y node_final
    results_astar = conn.runInstalledQuery("tg_astar", params={
        "source_vertex": node_final, "source_vertex.type": nodetype,
        "target_vertex": node_initial, "target_vertex.type": nodetype,
        "e_type_set": edgetype, "weight_type": "FLOAT",
        "latitude": "latitude", "longitude": "longitude",
        "weight_attribute": weight_attribute,
        "print_stats": "True"
    })

    key_csv_entrega = f"{node_initial}_{node_final}_{charge}"

    # Obtiene el orden de los nodos en el camino encontrado
    order_taken = path_taken(results_astar)
    
    # Obtiene el conjunto de aristas involucradas en el camino
    edge_set = results_astar[2]['@@display_edge_set']
    
    # Reorganiza las aristas de acuerdo al orden de los nodos en el camino
    reordered_edges_set = []
    for location in order_taken:
        for edge in edge_set:
            if edge['from_id'] == location:
                reordered_edges_set.append(edge)

    # Obtiene el conjunto de nodos involucrados en el camino
    node_set = results_astar[2]['tmp']
    
    # Reorganiza los nodos de acuerdo al orden en el camino
    reordered_nodes_set = []
    for location in order_taken:
        for item in node_set:
            if item['v_id'] == location:
                reordered_nodes_set.append(item)
    entregas_and_paths[key_csv_entrega] = reordered_nodes_set    # Guardamos todos los paths to check which path can give problem de sobrecarga . 
    try:
        # Recupera los atributos de los nodos inicial y final
        node_i_stock = reordered_nodes_set[0]["attributes"]["Stock"]
        node_f_stock = reordered_nodes_set[-1]["attributes"]["Stock"]
        node_i_unload_capacity = reordered_nodes_set[0]["attributes"]["UnloadCapacity"]
        node_f_load_capacity = reordered_nodes_set[-1]["attributes"]["LoadCapacity"]

        # Verifica si hay suficiente inventario en el nodo inicial
        with lock:
            if charge > node_i_stock:
                error_message = f"{node_initial} hasta {node_final}: No hay suficiente inventario ({charge} > {node_i_stock})\n"
                log_list += error_message

            # Verifica si hay suficiente capacidad de carga en el nodo final
            if charge > node_f_load_capacity:
                error_message = f"{node_initial} hasta {node_final}: No hay suficiente capacidad de carga en el almacén de recepción (charge: {charge} > LoadCapacity: {node_f_load_capacity})\n"
                log_list += error_message

            # Verifica si hay suficiente capacidad de descarga en el nodo inicial
            if charge > node_i_unload_capacity:
                error_message = f"{node_initial} hasta {node_final}: No hay suficiente capacidad de descarga desde el almacén de salida (charge: {charge} > UnloadCapacity: {node_i_unload_capacity})\n"
                log_list += error_message

            # Recupera la capacidad de la arista y el movimiento diario actual
            capacity_edge = reordered_edges_set[0]["attributes"]["Capacity"]
            daily_movement = reordered_edges_set[0]["attributes"]["Daily_movement"]

            # Verifica si la arista puede soportar el movimiento adicional de carga
            if daily_movement + charge > capacity_edge:
                error_message = f"Enviar {charge} palets desde {node_initial} hasta {node_final}: Capacidad de la arista insuficiente ({daily_movement + charge} > {capacity_edge})\n"
                log_list += error_message

        # Actualiza los inventarios de los nodos inicial y final
        nodes_batch = [
            (node_initial, {"Stock": node_i_stock - charge}),
            (node_final, {"Stock": node_f_stock + charge})
        ]

        # Actualiza el movimiento diario en las aristas recorridas y acumula el costo
        edges_batch = []
        with lock : 
            for element in reordered_edges_set:
                element['attributes']['Daily_movement'] += charge
                edges_batch.append((element["from_id"], element["to_id"], {"Daily_movement": element["attributes"]["Daily_movement"]}))
                palets_cost += element["attributes"]["Price"]

        # Añade los nodos inicial y final a los conjuntos de nodos únicos
        unique_origin_nodes.add(node_initial)
        unique_destination_nodes.add(node_final)

        return nodes_batch , edges_batch 

    # Manejo de excepciones en caso de que ocurra algún error durante el proceso
    except Exception as e:
        log_list += f"Error durante la transferencia de {node_initial} a {node_final}: {str(e)}\n"
        return [], []

    except Exception as e:
        error_message = f"Error en la transferancia: {e}\n"
        log_list += error_message
        return error_message


# Thread safety
lock = threading.Lock()


def make_daily_movements(input_file, column_origin, column_destination, column_transfert):
    # Variables globales para rastrear estadísticas y almacenar datos.
    global aristas_sobrecargadas,entregas_and_paths,operation_count, palets_number, palets_cost, unique_origin_nodes, unique_destination_nodes, log_list, nodes_batch_full, edges_batch_full, df
    operation_count = 0  # Conteo de operaciones realizadas.
    palets_number = 0  # Número total de palets transferidos.
    palets_cost = 0  # Costo total asociado a los palets transferidos.
    unique_origin_nodes = set()  # Conjunto de nodos de origen únicos.
    unique_destination_nodes = set()  # Conjunto de nodos de destino únicos.
    log_list = " "  # Registro de errores o mensajes importantes.
    nodes_batch_full = []  # Lista para almacenar nodos actualizados.
    edges_batch_full = []  # Lista para almacenar aristas actualizadas.
    aristas_sobrecargadas = []
    entregas_and_paths = {}


    # Lee el archivo CSV y carga los datos en un DataFrame.
    df = pd.read_csv(input_file)

    # Función para manejar la transferencia de volumen entre nodos.
    def perform_transfer(row):
        global log_list,operation_count, palets_number
        try:
            # Obtiene el origen, destino y volumen desde la fila del DataFrame.
            origin = str(row[column_origin])
            destination = str(row[column_destination])
            volume = float(row[column_transfert])
            
            # Llama a la función para obtener nodos y aristas actualizados.
            nodes_batch, edges_batch = transfert_nodes_and_edges(origin, destination, volume)
            
            # Usa un lock para garantizar que las actualizaciones a las listas sean seguras en un entorno multihilo.
            with lock:
                # Actualiza o agrega nodos y aristas a las listas correspondientes.
                add_or_update_final_nodes_batch(nodes_batch_full, nodes_batch, volume)
                add_or_update_final_edges_batch(edges_batch_full, edges_batch, volume)
            
            # Actualiza las métricas globales.
            operation_count += 1  # Incrementa el conteo de operaciones.
            palets_number += volume  # Incrementa el número total de palets.
            unique_origin_nodes.add(origin)  # Agrega el nodo de origen al conjunto de nodos únicos.
            unique_destination_nodes.add(destination)  # Agrega el nodo de destino al conjunto de nodos únicos.
        
        except Exception as e:
            # Registra un mensaje de error si ocurre una excepción.
            error_message = f"{origin} hasta {destination}, no camino posible.\n"
            log_list += error_message

    # Usa un ThreadPoolExecutor para procesar las transferencias en paralelo.
    with ThreadPoolExecutor(max_workers=1000) as executor:
        # Crea y envía tareas para cada fila del DataFrame.
        futures = [executor.submit(perform_transfer, row) for index, row in df.iterrows()]
        # Espera a que todas las tareas se completen.
        for future in as_completed(futures):
            pass  # Se pueden manejar los resultados de las tareas aquí si es necesario.

    # Actualiza los nodos y aristas en la base de datos.
    conn.upsertVertices("Nodes", nodes_batch_full)
    nodetype = "Nodes"
    edgetype = "distribute_to"
    conn.upsertEdges(sourceVertexType=nodetype, targetVertexType=nodetype, edgeType=edgetype, edges=edges_batch_full)
    
    # Llama a la función para actualizar el gráfico con el estado actual y obtiene el estado.
    state_log = actualize_graph_carga(conn)

    # Si no hubo errores, actualiza el mensaje de log para indicar éxito.
    if log_list == " ":
        log_list = "All operations were successful."

    print("Número de operaciones realizadas:", operation_count)
    print("Número total de palets transferidos:", palets_number)
    print("Costo total asociado a los palets transferidos:", palets_cost)
    print("Conjunto de nodos de origen únicos:", unique_origin_nodes)
    print("Conjunto de nodos de destino únicos:", unique_destination_nodes)
    print("Registro de errores o mensajes importantes:", log_list)
    print("Lista de nodos actualizados:", nodes_batch_full)
    print("Lista de aristas actualizadas:", edges_batch_full)
    print(f"log_list: {log_list}")
    print(f"state log : {state_log}")


    # Devuelve un conjunto de resultados incluyendo logs, un botón para recargar el mapa y KPIs actualizados.
    return (
        log_list,
        # reset_html(),  # Asume que esta función devuelve HTML para reiniciar la interfaz.
        gr.Button("Recargar mapa", elem_classes="otherbutton", visible=True),
        gr.Number(label="KPI : Numero de operaciones", value=operation_count, visible=True, elem_classes="dropdown"), 
        gr.Number(label="KPI : Numero de palets", value=palets_number, visible=True, elem_classes="dropdown"), 
        gr.Number(label="KPI : Numero de origen", value=len(unique_origin_nodes), visible=True, elem_classes="dropdown"),  
        gr.Number(label="KPI : Destinaciones diferentes", value=len(unique_destination_nodes), visible=True, elem_classes="dropdown"),
        gr.Number(label="KPI : Coste total", value=palets_cost, visible=True, elem_classes="dropdown"), 
        gr.Number(label="KPI : Coste por palet", value=round(palets_cost / 1, 1), visible=True, elem_classes="dropdown") 
    )

In [3]:
log_messages = make_daily_movements(r"C:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\base de datos graph\Tigergraph\CSV_entrega\entrega_demo_test.csv", "CODE_Origin", "CODE_Destination", "Palets")

Número de operaciones realizadas: 1
Número total de palets transferidos: 40.0
Costo total asociado a los palets transferidos: 33
Conjunto de nodos de origen únicos: {'Bilbao'}
Conjunto de nodos de destino únicos: {'Paris'}
Registro de errores o mensajes importantes:  Bilbao hasta Paris: No hay suficiente capacidad de carga en el almacén de recepción (charge: 40.0 > LoadCapacity: 19)
Bilbao hasta Paris: No hay suficiente capacidad de descarga desde el almacén de salida (charge: 40.0 > UnloadCapacity: 16)

Lista de nodos actualizados: [('Bilbao', {'Stock': 51.3}), ('Paris', {'Stock': 280.0})]
Lista de aristas actualizadas: [('Bilbao', 'Toulouse', {'Daily_movement': 40.0}), ('Toulouse', 'Clermont-Ferrand', {'Daily_movement': 40.0}), ('Clermont-Ferrand', 'Paris', {'Daily_movement': 40.0})]
log_list:  Bilbao hasta Paris: No hay suficiente capacidad de carga en el almacén de recepción (charge: 40.0 > LoadCapacity: 19)
Bilbao hasta Paris: No hay suficiente capacidad de descarga desde el almac

In [4]:
def extract_paths_from_deliveries(entregas_and_paths):
    # Initialiser un dictionnaire pour stocker les chemins extraits
    paths = {}

    # Parcourir chaque entrée dans le dictionnaire
    for key, delivery_data in entregas_and_paths.items():
        # Extraire les nœuds dans l'ordre du trajet
        path = [node['v_id'] for node in delivery_data]
        # Ajouter le chemin sous forme de tuple dans le dictionnaire des résultats
        paths[key] = tuple(path)
    
    return paths

# Appeler la fonction pour extraire les chemins
paths = extract_paths_from_deliveries(entregas_and_paths)

# Affichage des chemins extraits
print(paths)


{'Bilbao_Paris_40.0': ('Bilbao', 'Toulouse', 'Clermont-Ferrand', 'Paris')}


In [5]:
def get_overloaded_edges(aristas_sobrecargadas):
    # Créer un ensemble des arêtes surchargées (from_node, to_node, price)
    return {(arista['from_id'], arista['to_id'], arista['attributes']['Price']) for arista in aristas_sobrecargadas}

# Extraire les arêtes surchargées
overloaded_edges = get_overloaded_edges(aristas_sobrecargadas)

def extract_paths_from_deliveries(entregas_and_paths):
    # Initialiser un dictionnaire pour stocker les chemins extraits
    paths = {}

    # Parcourir chaque entrée dans le dictionnaire
    for key, delivery_data in entregas_and_paths.items():
        # Extraire les nœuds dans l'ordre du trajet
        path = [node['v_id'] for node in delivery_data]
        # Ajouter le chemin sous forme de tuple dans le dictionnaire des résultats
        paths[key] = tuple(path)
    
    return paths

# Appeler la fonction pour extraire les chemins
paths = extract_paths_from_deliveries(entregas_and_paths)


In [6]:
overloaded_edges

{('Clermont-Ferrand', 'Paris', 3),
 ('Clermont-Ferrand', 'Toulouse', 15),
 ('Paris', 'Clermont-Ferrand', 3),
 ('Toulouse', 'Clermont-Ferrand', 15)}

In [7]:
paths

{'Bilbao_Paris_40.0': ('Bilbao', 'Toulouse', 'Clermont-Ferrand', 'Paris')}

In [8]:
entregas_and_paths

{'Bilbao_Paris_40.0': [{'v_id': 'Bilbao',
   'v_type': 'Nodes',
   'attributes': {'LoadCapacity': 11.9,
    'UnloadCapacity': 16,
    'Stock': 91.3,
    'Carga': 'Optimal',
    'Capacity': 98.9,
    'latitude': 43.263,
    'longitude': -2.934,
    '@min_dist_heap': [{'dist': 33.90974, 'v': 'Toulouse'}],
    '@or_visited': True}},
  {'v_id': 'Toulouse',
   'v_type': 'Nodes',
   'attributes': {'LoadCapacity': 15,
    'UnloadCapacity': 13,
    'Stock': 72,
    'Carga': 'Optimal',
    'Capacity': 82,
    'latitude': 43.6047,
    'longitude': 1.4442,
    '@min_dist_heap': [{'dist': 18.90974, 'v': 'Clermont-Ferrand'}],
    '@or_visited': True}},
  {'v_id': 'Clermont-Ferrand',
   'v_type': 'Nodes',
   'attributes': {'LoadCapacity': 14,
    'UnloadCapacity': 11,
    'Stock': 65,
    'Carga': 'Optimal',
    'Capacity': 75,
    'latitude': 45.7824,
    'longitude': 3.0863,
    '@min_dist_heap': [{'dist': 3.553823, 'v': 'Paris'}],
    '@or_visited': True}},
  {'v_id': 'Paris',
   'v_type': 'Nodes

In [9]:
def split_path_into_edges(path):
    """Divise un chemin en arêtes entre chaque paire de nœuds adjacents."""
    return [(path[i], path[i + 1]) for i in range(len(path) - 1)]

def check_overloaded_paths(paths, overloaded_edges):
    """Vérifie les chemins surchargés en comparant les arêtes avec celles des chemins."""
    overloaded_segments = []

    # Convertir les arêtes surchargées en un set de tuples sans les prix pour la comparaison
    overloaded_edges_set = {(from_node, to_node) for from_node, to_node, price in overloaded_edges}

    # Parcourir tous les chemins dans le dictionnaire des chemins
    for path_key, path in paths.items():
        # Diviser le chemin en segments (paires de nœuds adjacents)
        path_edges = split_path_into_edges(path)

        # Vérifier chaque segment du chemin
        for edge in path_edges:
            if edge in overloaded_edges_set:
                overloaded_segments.append((path_key, edge))

    return overloaded_segments

# Exemple d'appel des fonctions avec vos données
overloaded_segments = check_overloaded_paths(paths, overloaded_edges)


In [10]:
overloaded_segments 

[('Bilbao_Paris_40.0', ('Toulouse', 'Clermont-Ferrand')),
 ('Bilbao_Paris_40.0', ('Clermont-Ferrand', 'Paris'))]

# Refaire la entrega 

In [11]:
csv = r"C:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\base de datos graph\Tigergraph\CSV_entrega\entrega_demo_test.csv"
# Get lines from csv of "overload segments"
df = pd.read_csv(csv)
# Liste pour stocker les DataFrames filtrés
filtered_rows = []

# Recherche des lignes correspondantes pour chaque élément de la liste
for element in overloaded_segments:
    name = element[0]  # Extraire le nom sous forme de chaîne
    # Extraire les informations d'origine, destination et palettes depuis name
    parts = name.split('_')
    origin = parts[0]
    destination = '_'.join(parts[1:-1])  # Rejoint les parties de destination (au cas où il y a des espaces)
    palets = float(parts[-1])  # Convertir le nombre de palettes en float

    # Filtrer les lignes correspondantes
    filtered_df = df[
        (df['CODE_Origin'] == origin) & 
        (df['CODE_Destination'] == destination) & 
        (df['Palets'] == palets)
    ]
    
    # Ajouter les lignes trouvées à la liste
    filtered_rows.append(filtered_df)

# Concaténer toutes les lignes filtrées
final_filtered_df = pd.concat(filtered_rows, ignore_index=True).drop_duplicates()

In [12]:
final_filtered_df

,ID,CODE_Origin,CODE_Destination,Palets
0,1,Bilbao,Paris,40


# Recuperer les chemins parcourus par A* des chemins parcourus

In [13]:
def erase_edge(source_vertex_id, target_vertex_id, vertex_type="Nodes", edgeType="distribute_to"):
    result = conn.delEdges(sourceVertexType=vertex_type,
                           targetVertexType=vertex_type,
                           edgeType=edgeType,
                           sourceVertexId=source_vertex_id,
                           targetVertexId=target_vertex_id)
    
    if result == {edgeType: 0}:
        message = f"Arista de {source_vertex_id} hacia {target_vertex_id} no existe o no ha podido borrarse\n"
    else:
        message = f"Arista de  {source_vertex_id} hacia {target_vertex_id} borrada \n"
    
    return message

In [14]:
def transfert_nodes_and_edges_reentrega_sequential(node_initial, node_final, charge, weight_attribute="Price", nodetype="Nodes", edgetype="distribute_to"):
    global palets_cost, log_list , entregas_and_paths
    edge_to_restore = []
    # Lanza la función A* para encontrar el camino óptimo entre node_initial y node_final
    results_astar = conn.runInstalledQuery("tg_astar", params={
        "source_vertex": node_final, "source_vertex.type": nodetype,
        "target_vertex": node_initial, "target_vertex.type": nodetype,
        "e_type_set": edgetype, "weight_type": "FLOAT",
        "latitude": "latitude", "longitude": "longitude",
        "weight_attribute": weight_attribute,
        "print_stats": "True"
    })


    key_csv_entrega = f"{node_initial}_{node_final}_{charge}"

    for element in results_astar[2]['@@display_edge_set']:
        # if element["attributes"]["Carga"] == "Sobrecargado":
        if charge + element["attributes"]["Daily_movement"] > element["attributes"]["Capacity"]:
            edge_to_restore.append(element)
            erase_edge(element["to_id"],element["from_id"])

    if not edge_to_restore:
        return

    results_astar = conn.runInstalledQuery("tg_astar", params={
            "source_vertex": node_final, "source_vertex.type": nodetype,
            "target_vertex": node_initial, "target_vertex.type": nodetype,
            "e_type_set": edgetype, "weight_type": "FLOAT",
            "latitude": "latitude", "longitude": "longitude",
            "weight_attribute": weight_attribute,
            "print_stats": "True"
        })
    
    edges_to_upsert = [
    (edge['from_id'], edge['to_id'], edge['attributes']) for edge in edge_to_restore
    ]

    conn.upsertEdges(
    sourceVertexType="Nodes",
    edgeType="distribute_to",
    targetVertexType="Nodes",
    edges=edges_to_upsert
    )


    # Obtiene el orden de los nodos en el camino encontrado
    order_taken = path_taken(results_astar)
    
    # Obtiene el conjunto de aristas involucradas en el camino
    edge_set = results_astar[2]['@@display_edge_set']
    
    # Reorganiza las aristas de acuerdo al orden de los nodos en el camino
    reordered_edges_set = []
    for location in order_taken:
        for edge in edge_set:
            if edge['from_id'] == location:
                reordered_edges_set.append(edge)

    # Obtiene el conjunto de nodos involucrados en el camino
    node_set = results_astar[2]['tmp']
    
    # Reorganiza los nodos de acuerdo al orden en el camino
    reordered_nodes_set = []
    for location in order_taken:
        for item in node_set:
            if item['v_id'] == location:
                reordered_nodes_set.append(item)
    entregas_and_paths[key_csv_entrega] = reordered_nodes_set 


       # Guardamos todos los paths to check which path can give problem de sobrecarga . 
    try:
        # Recupera los atributos de los nodos inicial y final
        node_i_stock = reordered_nodes_set[0]["attributes"]["Stock"]
        node_f_stock = reordered_nodes_set[-1]["attributes"]["Stock"]
        node_i_unload_capacity = reordered_nodes_set[0]["attributes"]["UnloadCapacity"]
        node_f_load_capacity = reordered_nodes_set[-1]["attributes"]["LoadCapacity"]

        # Verifica si hay suficiente inventario en el nodo inicial
        with lock:
            if charge > node_i_stock:
                error_message = f"{node_initial} hasta {node_final}: No hay suficiente inventario ({charge} > {node_i_stock})\n"
                log_list += error_message

            # Verifica si hay suficiente capacidad de carga en el nodo final
            if charge > node_f_load_capacity:
                error_message = f"{node_initial} hasta {node_final}: No hay suficiente capacidad de carga en el almacén de recepción (charge: {charge} > LoadCapacity: {node_f_load_capacity})\n"
                log_list += error_message

            # Verifica si hay suficiente capacidad de descarga en el nodo inicial
            if charge > node_i_unload_capacity:
                error_message = f"{node_initial} hasta {node_final}: No hay suficiente capacidad de descarga desde el almacén de salida (charge: {charge} > UnloadCapacity: {node_i_unload_capacity})\n"
                log_list += error_message

            # Recupera la capacidad de la arista y el movimiento diario actual
            capacity_edge = reordered_edges_set[0]["attributes"]["Capacity"]
            daily_movement = reordered_edges_set[0]["attributes"]["Daily_movement"]

            # Verifica si la arista puede soportar el movimiento adicional de carga
            if daily_movement + charge > capacity_edge:
                error_message = f"Enviar {charge} palets desde {node_initial} hasta {node_final}: Capacidad de la arista insuficiente ({daily_movement + charge} > {capacity_edge})\n"
                log_list += error_message

        # Actualiza los inventarios de los nodos inicial y final
        nodes_batch = [
            (node_initial, {"Stock": node_i_stock - charge}),
            (node_final, {"Stock": node_f_stock + charge})
        ]

        # Actualiza el movimiento diario en las aristas recorridas y acumula el costo
        edges_batch = []
        with lock : 
            for element in reordered_edges_set:
                element['attributes']['Daily_movement'] += charge
                edges_batch.append((element["from_id"], element["to_id"], {"Daily_movement": element["attributes"]["Daily_movement"]}))
                palets_cost += element["attributes"]["Price"]

        # Añade los nodos inicial y final a los conjuntos de nodos únicos
        unique_origin_nodes.add(node_initial)
        unique_destination_nodes.add(node_final)

        return nodes_batch , edges_batch 

    # Manejo de excepciones en caso de que ocurra algún error durante el proceso
    except Exception as e:
        log_list += f"Error durante la transferencia de {node_initial} a {node_final}: {str(e)}\n"
        return [], []

    except Exception as e:
        error_message = f"Error en la transferancia: {e}\n"
        log_list += error_message
        return error_message


# Thread safety
lock = threading.Lock()


def make_daily_movements_sequential_sobrecarga(input_file, column_origin, column_destination, column_transfert):
    # Variables globales para rastrear estadísticas y almacenar datos.
    global aristas_sobrecargadas,entregas_and_paths,operation_count, palets_number, palets_cost, unique_origin_nodes, unique_destination_nodes, log_list, nodes_batch_full, edges_batch_full, df
    operation_count = 0  # Conteo de operaciones realizadas.
    palets_number = 0  # Número total de palets transferidos.
    palets_cost = 0  # Costo total asociado a los palets transferidos.
    unique_origin_nodes = set()  # Conjunto de nodos de origen únicos.
    unique_destination_nodes = set()  # Conjunto de nodos de destino únicos.
    log_list = " "  # Registro de errores o mensajes importantes.
    nodes_batch_full = []  # Lista para almacenar nodos actualizados.
    edges_batch_full = []  # Lista para almacenar aristas actualizadas.
    aristas_sobrecargadas = []
    entregas_and_paths = {}

    # Lee el archivo CSV y carga los datos en un DataFrame.
    df = pd.read_csv(input_file)

    # Función para manejar la transferencia de volumen entre nodos.
    def perform_transfer(row):
        global log_list,operation_count, palets_number
        try:
            # Obtiene el origen, destino y volumen desde la fila del DataFrame.
            origin = str(row[column_origin])
            destination = str(row[column_destination])
            volume = float(row[column_transfert])
            
            # Llama a la función para obtener nodos y aristas actualizados.
            nodes_batch, edges_batch = transfert_nodes_and_edges_reentrega_sequential(origin, destination, volume)
            
            # Usa un lock para garantizar que las actualizaciones a las listas sean seguras en un entorno multihilo.
            with lock:
                # Actualiza o agrega nodos y aristas a las listas correspondientes.
                add_or_update_final_nodes_batch(nodes_batch_full, nodes_batch, volume)
                add_or_update_final_edges_batch(edges_batch_full, edges_batch, volume)
            
            # Actualiza las métricas globales.
            operation_count += 1  # Incrementa el conteo de operaciones.
            palets_number += volume  # Incrementa el número total de palets.
            unique_origin_nodes.add(origin)  # Agrega el nodo de origen al conjunto de nodos únicos.
            unique_destination_nodes.add(destination)  # Agrega el nodo de destino al conjunto de nodos únicos.
        
        except Exception as e:
            # Registra un mensaje de error si ocurre una excepción.
            print(e)
            error_message = f"{origin} hasta {destination}, no camino posible.\n"
            log_list += error_message

    for index, row in df.iterrows():
        actualize_graph_carga(conn)
        perform_transfer(row)

    # Actualiza los nodos y aristas en la base de datos.
    conn.upsertVertices("Nodes", nodes_batch_full)
    nodetype = "Nodes"
    edgetype = "distribute_to"
    conn.upsertEdges(sourceVertexType=nodetype, targetVertexType=nodetype, edgeType=edgetype, edges=edges_batch_full)
    
    # Llama a la función para actualizar el gráfico con el estado actual y obtiene el estado.
    state_log = actualize_graph_carga(conn)

    # Si no hubo errores, actualiza el mensaje de log para indicar éxito.
    if log_list == " ":
        log_list = "All operations were successful."

In [15]:
def launch_astar(node_final,node_initial):
    liste = []  # Liste pour stocker les résultats associés aux trajets
        # Exécution de la requête A* pour chaque ligne
    results_astar = conn.runInstalledQuery("tg_astar", params={
        "source_vertex": node_final,
        "source_vertex.type": "Nodes",
        "target_vertex": node_initial,
        "target_vertex.type": "Nodes",
        "e_type_set": "distribute_to",
        "weight_type": "FLOAT",
        "latitude": "latitude",
        "longitude": "longitude",
        "weight_attribute": "Price",
        "print_stats": "True"
    })
    
    # Association du trajet avec le résultat A* via un dictionnaire
    trajet_info = {
        'source': node_final,
        'destination': node_initial,
        'palets': palets,
        'edges': results_astar[2]['@@display_edge_set']  # Liste des arêtes associées
    }
    
    liste.append(trajet_info)  # Ajout du dictionnaire à la liste des trajets
    return liste

In [16]:
liste_to_undo = []  # Liste pour stocker les résultats associés aux trajets

for index, row in final_filtered_df.iterrows():
    # Extraction des valeurs de chaque ligne
    node_final = row['CODE_Origin']
    node_initial = row['CODE_Destination']
    palets = row['Palets']
    
    # Exécution de la requête A* pour chaque ligne
    results_astar = conn.runInstalledQuery("tg_astar", params={
        "source_vertex": node_final,
        "source_vertex.type": "Nodes",
        "target_vertex": node_initial,
        "target_vertex.type": "Nodes",
        "e_type_set": "distribute_to",
        "weight_type": "FLOAT",
        "latitude": "latitude",
        "longitude": "longitude",
        "weight_attribute": "Price",
        "print_stats": "True"
    })
    
    # Association du trajet avec le résultat A* via un dictionnaire
    trajet_info = {
        'source': node_final,
        'destination': node_initial,
        'palets': palets,
        'edges': results_astar[2]['@@display_edge_set']  # Liste des arêtes associées
    }
    
    liste_to_undo.append(trajet_info)  # Ajout du dictionnaire à la liste des trajets

In [17]:
liste_to_undo

[{'source': 'Bilbao',
  'destination': 'Paris',
  'palets': 40,
  'edges': [{'e_type': 'distribute_to',
    'from_id': 'Toulouse',
    'from_type': 'Nodes',
    'to_id': 'Bilbao',
    'to_type': 'Nodes',
    'directed': False,
    'attributes': {'Price': 15,
     'Carga': 'Optimal',
     'Capacity': 50,
     'Daily_movement': 40}},
   {'e_type': 'distribute_to',
    'from_id': 'Paris',
    'from_type': 'Nodes',
    'to_id': 'Clermont-Ferrand',
    'to_type': 'Nodes',
    'directed': False,
    'attributes': {'Price': 3,
     'Carga': 'Sobrecargado',
     'Capacity': 31,
     'Daily_movement': 40}},
   {'e_type': 'distribute_to',
    'from_id': 'Clermont-Ferrand',
    'from_type': 'Nodes',
    'to_id': 'Toulouse',
    'to_type': 'Nodes',
    'directed': False,
    'attributes': {'Price': 15,
     'Carga': 'Sobrecargado',
     'Capacity': 29,
     'Daily_movement': 40}}]}]

In [18]:
overloaded_segments

[('Bilbao_Paris_40.0', ('Toulouse', 'Clermont-Ferrand')),
 ('Bilbao_Paris_40.0', ('Clermont-Ferrand', 'Paris'))]

# AJOUTER UN ENVOI PARTIEL SI CA PASSE 

In [19]:
final_filtered_df

,ID,CODE_Origin,CODE_Destination,Palets
0,1,Bilbao,Paris,40


# Defaire les commandes qui surcharge

On prend element to undo et on lui quitte son nombre de palets respectif

In [20]:
def mettre_a_jour_daily_movement(trajets):
    # Parcours des trajets
    for trajet in trajets:
        palets = trajet['palets']  # Nombre de palets pour ce trajet
        
        # Parcours des arêtes pour chaque trajet
        for edge in trajet['edges']:
            # Mise à jour de 'Daily_movement' pour chaque arête
            edge['attributes']['Daily_movement'] -= palets
    #TODO : MANAGE STOCK 
    return trajets

trajets_mis_a_jour = liste_to_undo

# Appel de la fonction sur la liste trajets
trajets_mis_a_jour = mettre_a_jour_daily_movement(trajets_mis_a_jour)

In [21]:
trajets_mis_a_jour

[{'source': 'Bilbao',
  'destination': 'Paris',
  'palets': 40,
  'edges': [{'e_type': 'distribute_to',
    'from_id': 'Toulouse',
    'from_type': 'Nodes',
    'to_id': 'Bilbao',
    'to_type': 'Nodes',
    'directed': False,
    'attributes': {'Price': 15,
     'Carga': 'Optimal',
     'Capacity': 50,
     'Daily_movement': 0}},
   {'e_type': 'distribute_to',
    'from_id': 'Paris',
    'from_type': 'Nodes',
    'to_id': 'Clermont-Ferrand',
    'to_type': 'Nodes',
    'directed': False,
    'attributes': {'Price': 3,
     'Carga': 'Sobrecargado',
     'Capacity': 31,
     'Daily_movement': 0}},
   {'e_type': 'distribute_to',
    'from_id': 'Clermont-Ferrand',
    'from_type': 'Nodes',
    'to_id': 'Toulouse',
    'to_type': 'Nodes',
    'directed': False,
    'attributes': {'Price': 15,
     'Carga': 'Sobrecargado',
     'Capacity': 29,
     'Daily_movement': 0}}]}]

In [22]:
edges_to_upsert = [
    (edge['from_id'], edge['to_id'], edge['attributes']) 
    for trajet in trajets_mis_a_jour  # Parcours de chaque trajet
    for edge in trajet['edges']  # Parcours des arêtes associées à chaque trajet
]

result = conn.upsertEdges(
    sourceVertexType="Nodes",
    edgeType="distribute_to",
    targetVertexType="Nodes",
    edges=edges_to_upsert
)


In [23]:
STOP

NameError: name 'STOP' is not defined

In [ ]:
final_filtered_df

,ID,CODE_Origin,CODE_Destination,Palets
0,1,Santander,Murcie,35


In [ ]:
final_filtered_df.to_csv("lecsv_test.csv", index=False)
csv = r"C:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\base de datos graph\Tigergraph\Final_version\lecsv_test.csv"
make_daily_movements_sequential_sobrecarga(csv,"CODE_Origin", "CODE_Destination", "Palets")

# Funciones de modificacion de red

In [ ]:
import json
global logs_modif_network
logs_modif_network = " "


# ADD NODES 

def add_node(id, attributes_dict=None, vertex_type="Nodes"):
    global logs_modif_network
    if attributes_dict is None:
        conn1.upsertVertex(vertex_type, id)
        message =  f"Nodo {id} añadido\n"
        logs_modif_network +=message
        return logs_modif_network
    else:
        attributes = json.loads(attributes_dict)
        conn1.upsertVertex(vertex_type, id, attributes=attributes)
        message =  f"Nodo {id} añadido with attributos : {attributes} \n"
        logs_modif_network +=message
    return logs_modif_network

def get_values_nodes(*args):
    values = {key: val for key, val in zip(atributes_nodes.keys(), args)}
    return values

def get_values_edges(*args):
    values = {key: val for key, val in zip(atributes_edges.keys(), args)}
    return values["Capacity"]


def add_node_with_values(id, *args):
    attributes = get_values_nodes(*args)
    attributes_json = json.dumps(attributes)
    actualize_graph_carga(conn)
    return add_node(id, attributes_json), reset_html1()



# ADD EDGES 
def add_new_edge(source_vertex_id, target_vertex_id, attributes_dict=None,source_vertex_type="Nodes",target_vertex_type = "Nodes", edge_type="distribute_to"):
    global logs_modif_network
    if attributes_dict is None:
        conn1.upsertEdge(source_vertex_type, source_vertex_id, edge_type, target_vertex_type, target_vertex_id)
        actualize_graph_carga(conn)
        message =  f"Arista de {source_vertex_id} hasta {target_vertex_id}  añadida"
        logs_modif_network += message
        return logs_modif_network
    else:
        attributes = json.loads(attributes_dict)
        actualize_graph_carga(conn)
        conn1.upsertEdge(source_vertex_type, source_vertex_id, edge_type, target_vertex_type, target_vertex_id, attributes=attributes)
        message =  f"Arista de {source_vertex_id} hasta {target_vertex_id} añadida con los atributos siguientes: {attributes} \n"
        logs_modif_network += message
    return logs_modif_network

def add_edge_with_values(source_vertex_id, target_vertex_id, *args):
    attributes = get_values_edges(*args)
    attributes_json = json.dumps(attributes)
    return add_new_edge(source_vertex_id,target_vertex_id, attributes_json), reset_html1()

# GET ATTRIBUTES (for front, it takes the list of atributes to create a box for each atribute)
attributes_nodes_list = []
attributes_edges_list = []
conex_nodes_list = conn1.getVertexAttrs('Nodes')
conex_edges_list = conn1.getEdgeAttrs('distribute_to')

for node in conex_nodes_list:
    attributes_nodes_list.append(node[0])


for edge in conex_edges_list:
    attributes_edges_list.append(edge[0])


#Remove carga atributes as it is here only for a cheat with colors of the tigergraph map. no need to change that
if 'Carga' in attributes_nodes_list:
    attributes_nodes_list.remove('Carga')

if 'Carga' in attributes_edges_list:
    attributes_edges_list.remove('Carga')


# GET NODES NAMES
list_name_nodes = []
name_nodes_conn = conn1.getVertices("Nodes")
for i in range(0, len(name_nodes_conn)):
    list_name_nodes.append(name_nodes_conn[i]["v_id"])

def add_edge(source_vertex_id, target_vertex_id, attributes_dict=None, vertex_type="Nodes", edgeType="distribute_to"):
    global logs_modif_network
    if attributes_dict is None:
        conn1.upsertEdge(sourceVertexType=vertex_type,
                        targetVertexType=vertex_type,
                        edgeType=edgeType,
                        sourceVertexId=source_vertex_id,
                        targetVertexId=target_vertex_id)
        message =  f"Arista desde {source_vertex_id} hasta {target_vertex_id} añadido\n"
        logs_modif_network +=message
        return logs_modif_network
    else:
        attributes = json.loads(attributes_dict)
        conn1.upsertEdge(sourceVertexType=vertex_type,
                        targetVertexType=vertex_type,
                        edgeType=edgeType,
                        sourceVertexId=source_vertex_id,
                        targetVertexId=target_vertex_id,
                        attributes=attributes)
        message =  f"Arista {source_vertex_id} hasta {target_vertex_id} añadido con los atributos siguientes : {attributes} \n"
        logs_modif_network +=message
        return logs_modif_network


#Erase Data

def erase_edge(source_vertex_id, target_vertex_id, vertex_type="Nodes", edgeType="distribute_to"):
    global logs_modif_network
    result = conn1.delEdges(sourceVertexType=vertex_type,
                           targetVertexType=vertex_type,
                           edgeType=edgeType,
                           sourceVertexId=source_vertex_id,
                           targetVertexId=target_vertex_id)
    
    if result == {edgeType: 0}:
        message = f"Arista de {source_vertex_id} hacia {target_vertex_id} no existe o no ha podido borrarse\n"
    else:
        message = f"Arista de  {source_vertex_id} hacia {target_vertex_id} borrada \n"
    
    logs_modif_network += message
    return logs_modif_network , reset_html1()

    


def erase_node(id, vertex_type="Nodes"):
    global logs_modif_network
    result = conn1.delVerticesById(vertexIds=id, vertexType=vertex_type)
    
    if result == 0:
        message = f"Nodo {id} no existe o no ha podido borrarse\n"
    else:
        message = f"Nodo {id} borrado\n"
    
    logs_modif_network += message
    return logs_modif_network , reset_html1()




# add_edge_with_values("Las Palmas de Gran Canaria","Seville",{"Capacity": 0, "Daily_movement": 0, "Price": 0})
erase_edge("Seville","Madrid")


ConnectionError: HTTPSConnectionPool(host='ce7f96421a044abd9e8751d76a98bb8b.i.tgcloud.io', port=9000): Max retries exceeded with url: /version (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000015D1E59DB20>: Failed to resolve 'ce7f96421a044abd9e8751d76a98bb8b.i.tgcloud.io' ([Errno 11001] getaddrinfo failed)"))

# Tab modif cargas

In [ ]:
def increase_charge(conn, percentage):
    aristas = conn.getEdgesByType(edgeType="distribute_to")
    new_charges = []
    for arista in aristas:
        current_charge = arista["attributes"]["Daily_movement"]
        new_charge = current_charge * (1 + percentage / 100)
        arista["attributes"]["Daily_movement"] = new_charge
        new_charges.append((arista["from_id"], arista["to_id"], {"Daily_movement": new_charge}))

    return new_charges


def decrease_charge(conn, percentage):
    aristas = conn.getEdgesByType(edgeType="distribute_to")
    new_charges = []
    logs_state_aristas = ""
    arista_counts = {"Sobrecargado": 0, "Subcargado": 0, "Optimal": 0}

    for arista in aristas:
        current_charge = arista["attributes"]["Daily_movement"]
        capacity_edge = arista["attributes"]["Capacity"]
        new_charge = current_charge * (1 - percentage / 100)
        arista["attributes"]["Daily_movement"] = new_charge
        
        if capacity_edge == 0:
            nivel_arista = "No definido"
        else:
            if (new_charge / capacity_edge) > 1:
                nivel_arista = "Sobrecargado"
                arista_counts["Sobrecargado"] += 1
                logs_state_aristas += f"⚠️ La arista {arista['from_id']} hasta {arista['to_id']} es {nivel_arista} con un movimiento diario de {new_charge}\n"
            elif 0.7 < (new_charge / capacity_edge) <= 1:
                nivel_arista = "Optimal"
                arista_counts["Optimal"] += 1
            else:
                nivel_arista = "Subcargado"
                arista_counts["Subcargado"] += 1

        new_charges.append((arista["from_id"], arista["to_id"], {"Daily_movement": new_charge, "Carga": nivel_arista}))

    if logs_state_aristas == "":
        logs_state_aristas = "Aucune arista n'est surchargée."

    results = {
        "logs": logs_state_aristas,
        "arista_counts": arista_counts,
    }

    return results , new_charges

def update_batch_increase(conn,percentage):
    batch = increase_charge(conn, percentage)
    conn.upsertEdges(sourceVertexType="Nodes", targetVertexType="Nodes", edgeType="distribute_to", edges=batch)

def update_batch_decrease(conn,percentage):
    logs , batch = decrease_charge(conn, percentage)
    conn.upsertEdges(sourceVertexType="Nodes", targetVertexType="Nodes", edgeType="distribute_to", edges=batch)

    # return(gr.Textbox(value=logs["logs"], label="Estado de la red", visible=True,lines=2,max_lines=10, elem_classes="dropdown"),
    #     gr.Textbox(value=logs["node_counts"]["Sobrecargado"], label="Nodos sobrecargados", visible=True, elem_classes="dropdown"),
    #     gr.Textbox(value=logs["node_counts"]["Subcargado"], label="Nodos subcargados", visible=True, elem_classes="dropdown"),
    #     gr.Textbox(value=int(round(logs["arista_counts"]["Sobrecargado"]/2,0)), label="Aristas sobrecargadas", visible=True, elem_classes="dropdown"),  #Divide by 2 to take into account the fact that the edges are bidirectional
    #     gr.Textbox(value=int(round(logs["arista_counts"]["Subcargado"]/2,0)), label="Aristas subcargadas", visible=True, elem_classes="dropdown"))



# a = update_batch_decrease(conn,10)

In [ ]:
def make_daily_movements_with_percentages_positive(input_file, column_origin, column_destination, column_transfert,percentage):
    global operation_count3, palets_number3, palets_cost3, unique_origin_nodes, unique_destination_nodes, log_list, nodes_batch_full2,edges_batch_full2,df
    operation_count3 = 0
    palets_number3 = 0
    palets_cost3 = 0
    unique_origin_nodes = set()
    unique_destination_nodes = set()
    log_list = " "
    nodes_batch_full2 = []
    edges_batch_full2 = []

    df = pd.read_csv(input_file)

    def perform_transfer(row):
        global log_list
        try:
            origin = str(row[column_origin])
            destination = str(row[column_destination])
            positive_or_negative = "positive"
            if positive_or_negative == "positive":
                volume = float(row[column_transfert]) * (percentage / 100)
            if positive_or_negative == "negative":
                volume = float(row[column_transfert]) * (1 - percentage / 100)
            nodes_batch, edges_batch = transfert_nodes_and_edges1(origin, destination, volume)
            with lock:
                add_or_update_final_nodes_batch(nodes_batch_full2, nodes_batch, volume)
                add_or_update_final_edges_batch(edges_batch_full2, edges_batch, volume)
        except Exception as e:
            error_message = f"{origin} hasta {destination}, no camino posible.\n"
            log_list += error_message

    with ThreadPoolExecutor(max_workers=1000) as executor:
        futures = [executor.submit(perform_transfer, row) for index, row in df.iterrows()]
        for future in as_completed(futures):
            pass

    conn1.upsertVertices("Nodes", nodes_batch_full2)
    nodetype = "Nodes"
    edgetype = "distribute_to"
    conn1.upsertEdges(sourceVertexType=nodetype, targetVertexType=nodetype, edgeType=edgetype, edges=edges_batch_full2)
    results2 = actualize_graph_carga(conn1)

    if log_list == " ":
        log_list = "All operations were successful."

    return (reset_html1(),
        gr.Textbox(value=results2["logs"], label="Estado de la red", visible=True,lines=2,max_lines=10, elem_classes="dropdown"),
        gr.Textbox(value=results2["node_counts"]["Sobrecargado"], label="Nodos sobrecargados", visible=True, elem_classes="dropdown"),
        gr.Textbox(value=results2["node_counts"]["Subcargado"], label="Nodos subcargados", visible=True, elem_classes="dropdown"),
        gr.Textbox(value=int(round(results2["arista_counts"]["Sobrecargado"]/2,0)), label="Aristas sobrecargadas", visible=True, elem_classes="dropdown"),  #Divide by 2 to take into account the fact that the edges are bidirectional
        gr.Textbox(value=int(round(results2["arista_counts"]["Subcargado"]/2,0)), label="Aristas subcargadas", visible=True, elem_classes="dropdown")
        )
    # return (log_list, reset_html(), gr.Button("Recargar mapa",elem_classes="otherbutton",visible=True),
    #     gr.Number(label="KPI : Numero de operaciones", value = operation_count,visible=True,elem_classes="dropdown"), 
    #     gr.Number(label="KPI : Numero de palets",value = palets_number, visible=True,elem_classes="dropdown"), 
    #     gr.Number(label="KPI : Numero de origen",value = len(unique_origin_nodes), visible=True,elem_classes="dropdown"), 
    #     gr.Number(label="KPI : Destinaciones diferentes",value = len(unique_destination_nodes), visible=True,elem_classes="dropdown"), 
    #     gr.Number(label="KPI : Coste total",value = palets_cost, visible=True,elem_classes="dropdown"), 
    #     gr.Number(label="KPI : Coste por palet",value = round(palets_cost/palets_number,1), visible=True,elem_classes="dropdown"))

def make_daily_movements_with_percentages_negative(input_file, column_origin, column_destination, column_transfert,percentage):
    global operation_count3, palets_number3, palets_cost3, unique_origin_nodes, unique_destination_nodes, log_list, nodes_batch_full2,edges_batch_full2,df
    operation_count3 = 0
    palets_number3 = 0
    palets_cost3 = 0
    unique_origin_nodes = set()
    unique_destination_nodes = set()
    log_list = " "
    nodes_batch_full2 = []
    edges_batch_full2 = []

    df = pd.read_csv(input_file)

    def perform_transfer(row):
        global log_list
        try:
            origin = str(row[column_origin])
            destination = str(row[column_destination])
            positive_or_negative = "negative"
            if positive_or_negative == "positive":
                volume = float(row[column_transfert]) * (percentage / 100)
            if positive_or_negative == "negative":
                volume = float(row[column_transfert]) * (1 - percentage / 100)
            nodes_batch, edges_batch = transfert_nodes_and_edges1(origin, destination, volume)
            with lock:
                add_or_update_final_nodes_batch(nodes_batch_full2, nodes_batch, volume)
                add_or_update_final_edges_batch(edges_batch_full2, edges_batch, volume)
        except Exception as e:
            error_message = f"{origin} hasta {destination}, no camino posible.\n"
            log_list += error_message

    with ThreadPoolExecutor(max_workers=1000) as executor:
        futures = [executor.submit(perform_transfer, row) for index, row in df.iterrows()]
        for future in as_completed(futures):
            pass

    conn1.upsertVertices("Nodes", nodes_batch_full2)
    nodetype = "Nodes"
    edgetype = "distribute_to"
    conn1.upsertEdges(sourceVertexType=nodetype, targetVertexType=nodetype, edgeType=edgetype, edges=edges_batch_full2)
    results2 = actualize_graph_carga(conn1)

    if log_list == " ":
        log_list = "All operations were successful."

    return (reset_html1(),
        gr.Textbox(value=results2["logs"], label="Estado de la red", visible=True,lines=2,max_lines=10, elem_classes="dropdown"),
        gr.Textbox(value=results2["node_counts"]["Sobrecargado"], label="Nodos sobrecargados", visible=True, elem_classes="dropdown"),
        gr.Textbox(value=results2["node_counts"]["Subcargado"], label="Nodos subcargados", visible=True, elem_classes="dropdown"),
        gr.Textbox(value=int(round(results2["arista_counts"]["Sobrecargado"]/2,0)), label="Aristas sobrecargadas", visible=True, elem_classes="dropdown"),  #Divide by 2 to take into account the fact that the edges are bidirectional
        gr.Textbox(value=int(round(results2["arista_counts"]["Subcargado"]/2,0)), label="Aristas subcargadas", visible=True, elem_classes="dropdown")
        )

# a = make_daily_movements_with_percentages(r"C:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\base de datos graph\Tigergraph\CSV_entrega\df_1_move.csv", "CODE_Origin", "CODE_Destination", "Palets",10,"positive")

In [ ]:
def read_csv_path(input_file):
    df = pd.read_csv(input_file.name)
    target_column = sorted(list(df.columns))
    return  gr.Dropdown(choices = target_column) , gr.Dropdown(choices = target_column) , gr.Dropdown(choices = target_column)

Value first tab

In [ ]:
name_edge = "distribute_to"
name_node = "Nodes"


#######RED ORIGINAL######

numEdges = conn.getEdgeCount(name_edge)
numNodes = conn.getVertexCount(name_node)
edge = conn.getEdgeStats(name_edge)
avg_capacity = round(edge[name_edge]['Capacity']['AVG'],1)

edges = conn.getEdgesByType(name_edge)
total_capacity_edges = 0
for warehouse in edges :
    capacity_edges = warehouse["attributes"]["Capacity"]     
    total_capacity_edges += round(capacity_edges,1)

almacenes = conn.getVertices(name_node)
total_capacity_warehouse = 0
for warehouse in almacenes :
    capacity_edges = warehouse["attributes"]["Capacity"]      
    total_capacity_warehouse += round(capacity_edges,1)
    total_capacity_warehouse = round(total_capacity_warehouse,1)

almacen = conn.getVertices(name_node)
total_stock = 0
for warehouse in almacen :
    stock = warehouse["attributes"]["Stock"]     
    total_stock += stock
    total_stock = round(total_stock,0)

capacity_per_edge = round(total_capacity_warehouse/numEdges,1)


#######RED SIMULADA######
import concurrent.futures

def get_num_edges(conn, edge_type):
    return conn.getEdgeCount(edge_type)

def get_num_nodes(conn, node_type):
    return conn.getVertexCount(node_type)

def get_avg_capacity(conn, edge_type):
    edges = conn.getEdgeStats(edge_type)
    avg_capacity = edges[edge_type]['Capacity']['AVG']
    return round(avg_capacity/2, 1)

def get_total_capacity_edges(conn, edge_type):
    edges = conn.getEdgesByType(edge_type)
    total_capacity = 0
    for edge in edges:
        total_capacity += edge["attributes"]["Capacity"]
    return total_capacity /2

def get_total_capacity_nodes(conn, node_type):
    warehouses = conn.getVertices(node_type)
    total_capacity = 0
    for warehouse in warehouses:
        total_capacity += warehouse["attributes"]["Capacity"]
    return round(total_capacity, 1)

def get_total_stock(conn, node_type):
    warehouses = conn.getVertices(node_type)
    total_stock = 0
    for warehouse in warehouses:
        total_stock += warehouse["attributes"]["Stock"]
    return round(total_stock, 0)

def get_capacity_per_edge(total_capacity_warehouses, num_edges):
    if num_edges == 0:
        return 0
    return round(total_capacity_warehouses / num_edges, 0)

def get_KPI(conn, edge_type, node_type):
     with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {
            "num_edges": executor.submit(get_num_edges, conn, edge_type),
            "num_nodes": executor.submit(get_num_nodes, conn, node_type),
            "avg_capacity": executor.submit(get_avg_capacity, conn, edge_type),
            "total_capacity_edges": executor.submit(get_total_capacity_edges, conn, edge_type),
            "total_capacity_warehouses": executor.submit(get_total_capacity_nodes, conn, node_type),
            "total_stock": executor.submit(get_total_stock, conn, node_type)
        }

        results = {name: future.result() for name, future in futures.items()}
        results["capacity_per_edge"] = get_capacity_per_edge(results["total_capacity_warehouses"], results["num_edges"])
        
        return results

In [ ]:
import gradio as gr

class GradioMetro:
    
    def front_func(self):
        with gr.Blocks(css="stylessupersimple.css", title ="Red de distribución",theme="Monochrome") as demo:
            gr.Markdown("## ""![](file/Gradio/logo_lis.svg) Red de distribución""", elem_classes="cabecero")
            with gr.Tabs() as tabs:

                with gr.TabItem("KPIs red",id=1):
                    with gr.Row():
                        with gr.Column(scale=5):
                            map_tiger = gr.HTML(value = """ <html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, 
                                    initial-scale=1.0"></head><body><iframe width="1000" height="800"
                                    src=""" + tigergraph_insights_map1 + """ 
                                    title="tigergarph insights" frameborder="0"></iframe></body></html>""")
                            btn_reload_map =  gr.Button("Recargar mapa",elem_classes="otherbutton")
                            btn_reload_map.click(fn=reset_html1_1000,outputs=[map_tiger])  
                        with gr.Column(scale=2):
                            gr.Number(label="KPI : Cantidad de Aristas",value=numEdges,elem_classes='textbox')
                            gr.Number(label="KPI : Cantidad de Nodos",value = numNodes,elem_classes='textbox')
                            gr.Number(label= "KPI : Capacidad media por arista",value = avg_capacity,elem_classes='textbox')
                            gr.Number(label="KPI : Capacidad total de las aristas ",value = total_capacity_edges,elem_classes='textbox')
                            gr.Number(label= "KPI : Capacidad total de los Almacenes",value = total_capacity_warehouse,elem_classes='textbox')
                            gr.Number(label="KPI : Stock total de los almacenes",value = total_stock,elem_classes='textbox')
                
                with gr.TabItem("Cargar movimientos",id=2):
                    with gr.Row():
                        with gr.Column(scale=5):
                            upload_file = gr.File(label="Cargar fichero", type="filepath",elem_classes='textbox')

                        with gr.Column(scale=4):
                            dropdown_column_start = gr.Dropdown(label='Columna del envio', choices = [],  interactive=True, allow_custom_value= True,elem_classes='textbox')
                            dropdown_column_arrival = gr.Dropdown(label='Columna de la llegada', choices = [],  interactive=True, allow_custom_value= True,elem_classes='textbox')
                            dropdown_column_quantity = gr.Dropdown(label='Columna de la cantidad', choices = [],  interactive=True, allow_custom_value= True ,elem_classes='textbox')
                            load_data_send = gr.Button("Lanzar los envios",elem_classes="otherbutton")
                            upload_file.upload(read_csv_path,inputs=[upload_file],outputs=[dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity])
                    with gr.Row():
                        logs = gr.Textbox(label="Logs",elem_classes="dropdown")

                    with gr.Row():
                        with gr.Column(scale=10):
                            map_entrega = gr.HTML(value = """ <html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, 
                                    initial-scale=1.0"></head><body><iframe width="800" height="800"
                                    src=""" + tigergraph_insights_map + """ 
                                    title="tigergarph insights" frameborder="0"></iframe></body></html>""", visible = False
                                    )
                            btn_reload_map2 = gr.Button("Recargar mapa",elem_classes="otherbutton",visible=False)
                            btn_reload_map2.click(fn=reset_html,outputs=[map_entrega])  
                        with gr.Column(scale=6):
                  
                            operation_number = gr.Number(label="KPI : Numero de operaciones", visible=False)
                            palets_number = gr.Number(label="KPI : Numero de palets", visible=False)
                            origin_number = gr.Number(label="KPI : Numero de origen", visible=False)
                            distinct_destinations = gr.Number(label="KPI : Destinaciones diferentes", visible=False)
                            total_cost = gr.Number(label="KPI : Coste total", visible=False)
                            total_cost_per_palet = gr.Number(label="KPI : Coste por palet", visible=False)
        
                        load_data_send.click(make_daily_movements,inputs=[upload_file,dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity],
                                             outputs=[logs,map_entrega,btn_reload_map2,operation_number,palets_number,origin_number,distinct_destinations,total_cost,total_cost_per_palet])

                with gr.TabItem("Escenario What if", id=3):
                    with gr.Row():
                        with gr.Column():
                            gr.Markdown("# Añadir/modificar nodo",elem_classes="cabecero")
                            new_node_id = gr.Textbox(label="Nombre nuevo nodo",elem_classes="dropdown")
                            for e in attributes_nodes_list:
                                with gr.Row():
                                    atributes_nodes[e] = gr.Number(label=e,elem_classes="dropdown")
                        
                            with gr.Row():
                                btn_modif_nodes = gr.Button("Añadir/modificar nodos",elem_classes="otherbutton")

                        with gr.Column():
                            gr.Markdown("# Añadir/modificar arista",elem_classes="cabecero")

                            source_vertex_id_edge = gr.Textbox(label="Nodo de origen",elem_classes="dropdown",elem_id = "source_vertex")
                            target_vertex_id_edge = gr.Textbox(label="Nodo de destino",elem_classes="dropdown")

                            for e in attributes_edges_list:
                                with gr.Row():
                                    atributes_edges[e] = gr.Number(label=e,elem_classes="dropdown")
                            btn_modif_edge = gr.Button("Añadir/modificar arista",elem_classes="otherbutton")

                        with gr.Column():
                            gr.Markdown("# Borrar nodo/arista",elem_classes="cabecero")
                            source_vertex_id = gr.Textbox(label="Nodo de origen",elem_classes="dropdown",container = True)
                            target_vertex_id = gr.Textbox(label="Nodo de destino (si es una arista)",elem_classes="dropdown",container = True)
                            btn_erase_node = gr.Button("Borrar nodo",elem_classes="otherbutton")
                            btn_erase_edge = gr.Button("Borrar arista",elem_classes="otherbutton")  

                    with gr.Row():
                        logs = gr.Textbox(label = "logs",elem_classes="dropdown")

                    with gr.Row():
                        with gr.Column(scale=5):
                            upload_file = gr.File(label="Cargar fichero", type="filepath",elem_classes='textbox')
                        with gr.Column(scale=4):
                            dropdown_column_start = gr.Dropdown(label='Columna del envio', choices = [],  interactive=True, allow_custom_value= True, elem_classes="dropdown")
                            dropdown_column_arrival = gr.Dropdown(label='Columna de la llegada', choices = [],  interactive=True, allow_custom_value= True, elem_classes="dropdown")
                            dropdown_column_quantity = gr.Dropdown(label='Columna de la cantidad', choices = [],  interactive=True, allow_custom_value= True, elem_classes="dropdown")
                            load_data_send1 = gr.Button("Lanzar los envios en la red modificada",elem_classes="otherbutton")
                            upload_file.upload(read_csv_path,inputs=[upload_file],outputs=[dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity])

                    with gr.Row():
                        logs1 = gr.Textbox(label = "logs",elem_classes="dropdown")

                            
                    with gr.Row():
                        with gr.Column():
                            markdown1 = gr.Markdown("# KPIs red original",visible=False,elem_classes = "cabecero")
                            num_edges_number1 = gr.Number(label="KPI : Cantidad de Aristas",value=numEdges,visible=False)
                            num_nodes_number1 = gr.Number(label="KPI : Cantidad de Nodos",value = numNodes,visible=False)
                            capacity_average_number1 = gr.Number(label= "KPI : Capacidad media por arista",value = avg_capacity,visible=False)
                            total_capacity_edge_number1 = gr.Number(label="KPI : Capacidad total de las aristas ",value = total_capacity_edges,visible=False)
                            total_capacity_warehouse_number1 = gr.Number(label= "KPI : Capacidad total de los Almacenes",value = total_capacity_warehouse,visible=False)
                            total_stock_number1 = gr.Number(label="KPI : Stock total de los almacenes",value = total_stock,visible=False)
                            ####KPI ENTREGA RED ORGINAL####
                        with gr.Column():
                            markdown2 = gr.Markdown("# KPIs Entrega original",visible=False,elem_classes = "cabecero")
                            operation_number1 = gr.Number(label= "KPI : Numero de operaciones",visible=False)
                            palets_number1 = gr.Number(label= "KPI : Numero de palets",visible=False)
                            origin_number1 = gr.Number(label= "KPI : Numero de origen",visible=False)
                            distinct_destinations1 = gr.Number(label= "KPI : Destinaciones diferentes",visible=False)
                            total_cost1 = gr.Number(label= "KPI : Coste total",visible=False)
                            total_cost_per_palet1 = gr.Number(label= "KPI : Coste por palet",visible=False)
                        with gr.Column():
                            ###KPI RED MODIFICADA####
                            markdown1_simul =gr.Markdown("# KPIs red modificada",visible=False,elem_classes = "cabecero")
                            num_edges_number_simul = gr.Number(label="KPI : Cantidad de Aristas",value="numEdges1",visible=False)
                            num_nodes_number_simul = gr.Number(label="KPI : Cantidad de Nodos",value = "numNodes1",visible=False)
                            capacity_average_number_simul = gr.Number(label= "KPI : Capacidad media por arista",value = "avg_capacity1",visible=False)
                            total_capacity_edge_number_simul = gr.Number(label="KPI : Capacidad total de las aristas ",value = "total_capacity_edges1",visible=False)
                            total_capacity_warehouse_number_simul = gr.Number(label= "KPI : Capacidad total de los Almacenes",value = "total_capacity_warehouse1",visible=False)
                            total_stock_number_simul = gr.Number(label="KPI : Stock total de los almacenes",value = "total_stock1",visible=False)
                        with gr.Column():
                            markdown2_simul =gr.Markdown("# KPIs entrega red modificada",visible=False,elem_classes = "cabecero")
                            operation_number_simul = gr.Number(label= "KPI : Numero de operaciones",visible=False)
                            palets_number_simul = gr.Number(label= "KPI : Numero de palets",visible=False)
                            origin_number_simul = gr.Number(label= "KPI : Numero de origen",visible=False)
                            distinct_destinations_simul = gr.Number(label= "KPI : Destinaciones diferentes",visible=False)
                            total_cost_simul = gr.Number(label= "KPI : Coste total",visible=False)
                            total_cost_per_palet_simul = gr.Number(label= "KPI : Coste por palet",visible=False)
                    
                    with gr.Row():
                        diference_price = gr.Number(label=" porcentaje diferencia precio",value = avg_capacity,visible=False)



                    with gr.Row():
                        with gr.Column():
                            markdown_mapa_original =gr.Markdown("# Mapa red original",visible=False,elem_classes = "cabecero")
                            map_entrega_original = gr.HTML(value = """ <html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, 
                                        initial-scale=1.0"></head><body><iframe width="800" height="800"
                                        src=""" + tigergraph_insights_map + """ 
                                        title="tigergarph insights" frameborder="0"></iframe></body></html>""",visible=False)
                            btn_reload_map_original = gr.Button("Recargar mapa",visible=False)

                        with gr.Column():
                            markdown_mapa_modificada =gr.Markdown("# Mapa modificada",visible=False,elem_classes = "cabecero")
                            map_entrega_simulacion = gr.HTML(value = """ <html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, 
                                        initial-scale=1.0"></head><body><iframe width="800" height="800"
                                        src=""" + tigergraph_insights_map1 + """ 
                                        title="tigergarph insights" frameborder="0"></iframe></body></html>""",visible=False)
                            btn_reload_map_simul = gr.Button("Recargar mapa",visible=False)
                    
                    with gr.Row():
                        valor_porcentaje = gr.Number(label="Porcentaje de subir/bajar la carga",visible=False)
                        
                    with gr.Row():
                        btn_subir_carga = gr.Button("Subir carga",elem_classes="otherbutton",visible=False)
                        btn_bajar_carga = gr.Button("Bajar carga",elem_classes="otherbutton",visible=False)

                    with gr.Row():
                        with gr.Column(scale=5):
                            logs_red_state_o = gr.Textbox(label="Estado de la red", visible=False,elem_classes="dropdown")

                            KPIo_node_sobrecargado = gr.Textbox(label="Nodos sobrecargados", visible=False,elem_classes="dropdown")
                            KPIo_node_subcargado = gr.Textbox(label="Nodos subcargados", visible=False,elem_classes="dropdown")
                            KPIo_arista_sobrecargada = gr.Textbox(label="Aristas sobrecargadas", visible=False,elem_classes="dropdown")
                            KPIo_arista_subcargada = gr.Textbox(label="Aristas subcargadas", visible=False,elem_classes="dropdown")
                        with gr.Column(scale=5):
                            logs_red_state_m = gr.Textbox(label="Estado de la red", visible=False,elem_classes="dropdown")
                            KPIm_node_sobrecargado = gr.Textbox(label="Nodos sobrecargados", visible=False,elem_classes="dropdown")
                            KPIm_node_subcargado = gr.Textbox(label="Nodos subcargados", visible=False,elem_classes="dropdown")
                            KPIm_arista_sobrecargada = gr.Textbox(label="Aristas sobrecargadas", visible=False,elem_classes="dropdown")
                            KPIm_arista_subcargada = gr.Textbox(label="Aristas subcargadas", visible=False,elem_classes="dropdown")


                        load_data_send1.click(make_daily_movements1,inputs=[upload_file,dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity],
                                            outputs=[logs1,markdown_mapa_original,map_entrega_original,btn_reload_map_original,markdown_mapa_modificada,map_entrega_simulacion,btn_reload_map_simul,
                                                    markdown1,num_edges_number1,num_nodes_number1,capacity_average_number1,total_capacity_edge_number1,total_capacity_warehouse_number1,total_stock_number1,   #Red original
                                                    markdown2,operation_number1,palets_number1,origin_number1,distinct_destinations1,total_cost1,total_cost_per_palet1,   #Entrega original
                                                    markdown1_simul,num_edges_number_simul,num_nodes_number_simul,capacity_average_number_simul,total_capacity_edge_number_simul,total_capacity_warehouse_number_simul,total_stock_number_simul, #Red simulada
                                                    markdown2_simul,operation_number_simul,palets_number_simul,origin_number_simul,distinct_destinations_simul,total_cost_simul,total_cost_per_palet_simul #Entrega simulada 
                                                    ,logs_red_state_o,
                                                    KPIo_node_sobrecargado,KPIo_node_subcargado,KPIo_arista_sobrecargada,KPIo_arista_subcargada,
                                                    logs_red_state_m,
                                                    KPIm_node_sobrecargado,KPIm_node_subcargado,KPIm_arista_sobrecargada,KPIm_arista_subcargada,diference_price,
                                                    valor_porcentaje,btn_subir_carga,btn_bajar_carga],
                                                    scroll_to_output=True) 
                        
                        btn_subir_carga.click(make_daily_movements_with_percentages_positive,inputs=[upload_file,dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity,valor_porcentaje],
                                              outputs=[map_entrega_simulacion,logs_red_state_m,KPIm_node_sobrecargado,KPIm_node_subcargado,KPIm_arista_sobrecargada,KPIm_arista_subcargada])
                        btn_bajar_carga.click(make_daily_movements_with_percentages_negative,inputs=[upload_file,dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity,valor_porcentaje],
                                              outputs=[map_entrega_simulacion,logs_red_state_m,KPIm_node_sobrecargado,KPIm_node_subcargado,KPIm_arista_sobrecargada,KPIm_arista_subcargada])


                    inputs_edge = [source_vertex_id_edge,target_vertex_id_edge] + [atributes_edges[e] for e in attributes_edges_list]
                    
                    inputs_nodes = [new_node_id] + [atributes_nodes[e] for e in attributes_nodes_list]
                    
                    def add_edge_with_values_wrapper(source, target, *args):
                        attributes = {e: arg for e, arg in zip(attributes_edges_list, args)}
                        return add_edge_with_values(source, target, attributes)

                    btn_modif_nodes.click(fn=add_node_with_values, inputs=inputs_nodes, outputs=[logs,map_entrega_simulacion])
                    btn_modif_edge.click(fn=add_edge_with_values_wrapper, inputs=inputs_edge, outputs=[logs,map_entrega_simulacion])
                    btn_erase_node.click(fn=erase_node, inputs=[source_vertex_id], outputs=[logs,map_entrega_simulacion])
                    btn_erase_edge.click(fn=erase_edge, inputs=[source_vertex_id,target_vertex_id], outputs=[logs,map_entrega_simulacion])
                    btn_reload_map_original.click(fn=reset_html_800,outputs=[map_entrega_original])
                    btn_reload_map_simul.click(fn=reset_html1_800,outputs=[map_entrega_simulacion])

                demo.launch(max_threads=1000)
                
if __name__ == "__main__":

    lis = GradioMetro()
    lis.front_func()


    #Reactualizar los colores cuando se hace modif de arrista despues de la entrega

c:\Users\JulienRigot\Projets\zzEnvs\Transport\Lib\site-packages\gradio\routes.py:1150: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
c:\Users\JulienRigot\Projets\zzEnvs\Transport\Lib\site-packages\fastapi\applications.py:4495: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  return self.router.on_event(event_type)


NameError: name 'tigergraph_insights_map1' is not defined